In [ ]:
%%capture

%cd ..


In [ ]:
from datasets import load_dataset

from similarity_augmentations import consts

mmlu = load_dataset(consts.MMLU_HF_PATH, name="all")
print(mmlu)

mmlu_train_ctx = mmlu["auxiliary_train"]["question"]
print(len(mmlu_train_ctx))
print(mmlu["auxiliary_train"][0]["question"])
print(mmlu_train_ctx[0])


Found cached dataset mmlu (/home/mgallo/.cache/huggingface/datasets/cais___mmlu/all/1.0.0/49beb454d1ef6d7994f43183dc194713f39dd672b4f5a92c08b0b394991910c7)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
})
99842
Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were
Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw B

In [ ]:
mutual = load_dataset(consts.MUTUAL_HF_PATH, name="mutual_plus")
print(mutual)

mutualplus_train_ctx = mutual["train"]["article"]
print(len(mutualplus_train_ctx))
print(mutualplus_train_ctx[0])


Found cached dataset mutual_harness (/home/mgallo/.cache/huggingface/datasets/lighteval___mutual_harness/mutual_plus/0.0.1/be31c67b35f0d5c4cce450a4e8475f0eb26ae243f5bfd701a524305282b0873a)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['answers', 'options', 'article', 'id'],
        num_rows: 7088
    })
    test: Dataset({
        features: ['answers', 'options', 'article', 'id'],
        num_rows: 886
    })
    validation: Dataset({
        features: ['answers', 'options', 'article', 'id'],
        num_rows: 886
    })
})
7088
M: Excuse me. How much is this suit? F: It's on sale today for $750. It's normally $900. M: Wow, that is pretty expensive! I was thinking that it might be 4 or 500. F: This material is imported from Italy. It's the finest in the world, and if you bought a suit made of this material at many department stores, you would pay about $2000. M: Uh-hah. But isn't that the point of coming to a market like this, to get a discount compared to the expensive department stores? Besides I saw a suit just like this one a few stalls down, and they were selling it for $600. I still thought that it was too expensive.


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


from similarity_augmentations import consts


embedder = HuggingFaceEmbeddings(model_name=consts.DEFAULT_EMBEDDING_MODEL)
embedder


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='all-distilroberta-v1', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [ ]:
from similarity_augmentations import conf
from similarity_augmentations.embedding import crud

mutualplus_db = crud.create_or_load_faiss(
    conf.VECTORDB_DIR,
    "mutual_plus__train__all-distilroberta-v1",
    embedder,
    dataset=mutualplus_train_ctx,
)
print(mutualplus_db)
mutualplus_db.index, mutualplus_db.index.ntotal


[2023-10-09 02:12:59,027: INFO] Loading FAISS index 'mutual_plus__train__all-distilroberta-v1' from 'vector_db' (similarity_augmentations.embedding.crud:55)


<langchain.vectorstores.faiss.FAISS object>


(<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *'> >,
 7088)

**NOTE** Pay attention, in MuTual some `article`s are the same! They differ in the `options` they offer, but we do not 
encode this or consider it for similarity search :/

In [ ]:
mutual["train"]["article"][10:13]


["M: Good evening, what can I do for you? F: Good evening. I would like to buy a bottle of perfume. M: OK. We have many kinds of perfume here. You can choose freely. F: Which brand would you recommend? I don't want to buy one with quality problems. M: Don't worry, we have no such problems. How about this one? It sells very well. F: Umm, it smells good. How much is it? M: 400 yuan per bottle. And if you buy 2 bottles at the same time, you can get a 10% discount. We have other special offers as well. If you buy lipstick or face cream, you can get a free gift. F: Thanks. I just need perfume when I want to buy lipstick I will come here again.",
 "M: Good evening, what can I do for you? F: Good evening. I would like to buy a bottle of perfume. M: OK. We have many kinds of perfume here. You can choose freely. F: Which brand would you recommend? I don't want to buy one with quality problems. M: Don't worry, we have no such problems. How about this one? It sells very well. F: Umm, it smells go

In [ ]:
mutual["train"]["options"][10:13]


[['M: Sure. It was 400 yuan each bottle early, but now, it is 360 for one. Thank you.',
  'M: Sure. It will be 400 yuan for one bottle. Thank you.',
  'M: I’m sorry, I didn’t understand. Could you repeat a little louder, please?',
  'M: Sure. It will be 440 yuan for one bottle. Thank you.'],
 ['M: OK. Just one bottle of perfume, right?',
  'M: I’m sorry, I didn’t understand. Could you repeat a little louder, please?',
  'M: Sure. One bottle of perfume and a lipstick. It will be 400 yuan in total.',
  'M: Sure. One bottle of face cream, right?'],
 ['M: Sure. Thanks for coming to our store. Here are your perfume and lipstick.',
  'M: Sure. Thanks for coming to our store.',
  'M: I didn’t hear you. Please could you tell me again?',
  'M: Please finish your work on time.']]

In [ ]:
mmlu_db = crud.create_or_load_faiss(
    conf.VECTORDB_DIR,
    "mmlu__auxiliary_train__all-distilroberta-v1",
    embedder,
    dataset=mmlu_train_ctx,
    batch_size=int(1e4),
)
print(mmlu_db)
mmlu_db.index, mmlu_db.index.ntotal


[2023-10-09 02:13:00,399: INFO] Loading FAISS index 'mmlu__auxiliary_train__all-distilroberta-v1' from 'vector_db' (similarity_augmentations.embedding.crud:55)
<langchain.vectorstores.faiss.FAISS object>


(<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *'> >,
 99842)

In [ ]:
# import numpy as np

# query_idx = np.random.randint(0, len(mutualplus_train_ctx) + 1)
query_idx = 2514
print(query_idx, mmlu_db.index.reconstruct(query_idx).shape)
print(mutualplus_train_ctx[query_idx])
print(mutualplus_db.docstore.search(mutualplus_db.index_to_docstore_id[query_idx]))

mmlu_db.similarity_search_with_score_by_vector(
    mutualplus_db.index.reconstruct(query_idx).tolist()
)


2514 (768,)
M: Hello, my dear, you look sad. What happened? F: Uh, hello. My name is Sally, Sally Miller. I can't find my passport. I got it last month. It's my first one you know. I have already reported it to the police and they said I should come here. As anyone found it yet? M: What did you say? What was your name? F: Miller, Sally Miller. M: Oh, you are in the book. I think I can help you.
page_content="M: Hello, my dear, you look sad. What happened? F: Uh, hello. My name is Sally, Sally Miller. I can't find my passport. I got it last month. It's my first one you know. I have already reported it to the police and they said I should come here. As anyone found it yet? M: What did you say? What was your name? F: Miller, Sally Miller. M: Oh, you are in the book. I think I can help you." metadata={'idx': 2514}


[(Document(page_content='Nora is an American girl. She studies in a middle school. She has a little sister. Her name is Kate. Kate is only four. Nora likes Kate very much. Today is Sunday. Nora wants her pen. She takes out her pencil case. "Oh, dear! Where\'s my pen?" She can\'t find her pen. She goes to ask her sister in her bedroom. "Kate! I can\'t find my pen. Can you see......Oh, what are you doing with   my pen?" "I\'m writing to my friend, Betty." Kate answers. "But how can you? You don\'t know what to write."."It doesn\'t matter  . Betty can\'t read." Kate says. Where is Kate?    _', metadata={'idx': 85360}),
  0.99581957),
 (Document(page_content="M: Hi, Ann! Tomorrow will be August 1st. When is your holiday going to start? W: Very soon. I'm going to leave the day after tomorrow. M: How long are you going to be away? W: About two weeks. M: Where are you going? W: I'm going to the beach this time. M: Are you going to stay alone? W: No. I will stay in a hotel with my friends. M: 

**NOTE** 2 things:
1. There are many very similar articles in MMLU as well.
2. MMLU also contain dialogues, and if these are often retrieved by similarity, removing speaker 
   tags doesn't really affect results much...

In [ ]:
from similarity_augmentations.finetuning import augmentations

p = 0.5
scaled_p = len(mutualplus_train_ctx) * p / len(mmlu_train_ctx)
print(p, round(scaled_p, 3))
n_samples = augmentations.to_mmlu_size(scaled_p, len(mmlu_train_ctx))
n_samples


0.5 0.035


3544

In [ ]:
from tqdm.autonotebook import tqdm

sampled = set()
closest_mmlu_points = {}
closest_dict = {}

for i, mutual_dp in tqdm(enumerate(mutualplus_train_ctx), total=len(mutualplus_train_ctx)):
    found, k = False, 1
    while not found and k <= mmlu_db.index.ntotal:
        mmlu_closest = mmlu_db.similarity_search_with_score_by_vector(
            mutualplus_db.index.reconstruct(i).tolist(), k=k
        )
        kth_closest_doc, kth_distance = mmlu_closest[k - 1]
        kth_closest_mmlu_idx = kth_closest_doc.metadata["idx"]
        if i not in closest_dict:
            closest_dict[i] = kth_closest_mmlu_idx
        if kth_closest_mmlu_idx not in sampled:
            sampled.add(kth_closest_mmlu_idx)
            closest_mmlu_points[i] = {
                "idx": kth_closest_mmlu_idx,
                "rank": k,
                "distance": kth_distance,
            }
            found = True
        else:
            k += 1


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
closest_mmlu_points


{0: {'idx': 42541, 'rank': 1, 'distance': 0.9699097},
 1: {'idx': 79730, 'rank': 1, 'distance': 0.8783765},
 2: {'idx': 73972, 'rank': 1, 'distance': 0.80798745},
 3: {'idx': 77756, 'rank': 1, 'distance': 0.81840813},
 4: {'idx': 79019, 'rank': 1, 'distance': 0.86363363},
 5: {'idx': 71768, 'rank': 1, 'distance': 0.9946202},
 6: {'idx': 71766, 'rank': 1, 'distance': 0.7166224},
 7: {'idx': 88821, 'rank': 1, 'distance': 0.8657171},
 8: {'idx': 77962, 'rank': 1, 'distance': 0.73718405},
 9: {'idx': 72130, 'rank': 1, 'distance': 0.8492526},
 10: {'idx': 82902, 'rank': 1, 'distance': 0.7559252},
 11: {'idx': 82900, 'rank': 2, 'distance': 0.7593404},
 12: {'idx': 82899, 'rank': 3, 'distance': 0.7646302},
 13: {'idx': 89993, 'rank': 1, 'distance': 1.0443623},
 14: {'idx': 79020, 'rank': 1, 'distance': 1.0587243},
 15: {'idx': 48032, 'rank': 1, 'distance': 0.9345333},
 16: {'idx': 48034, 'rank': 2, 'distance': 0.9732434},
 17: {'idx': 87426, 'rank': 1, 'distance': 0.87675095},
 18: {'idx': 87

In [ ]:
dict(sorted(closest_mmlu_points.items(), key=lambda v: v[1]["distance"]))


{862: {'idx': 87178, 'rank': 1, 'distance': 0.39217412},
 863: {'idx': 79752, 'rank': 3, 'distance': 0.43426543},
 861: {'idx': 87380, 'rank': 1, 'distance': 0.4701337},
 1621: {'idx': 13347, 'rank': 1, 'distance': 0.48573753},
 800: {'idx': 47989, 'rank': 3, 'distance': 0.5116639},
 799: {'idx': 47991, 'rank': 2, 'distance': 0.520386},
 60: {'idx': 79865, 'rank': 1, 'distance': 0.5209559},
 61: {'idx': 79863, 'rank': 2, 'distance': 0.5212731},
 798: {'idx': 47990, 'rank': 1, 'distance': 0.53459895},
 1385: {'idx': 68732, 'rank': 2, 'distance': 0.5475934},
 202: {'idx': 50411, 'rank': 2, 'distance': 0.5498006},
 1843: {'idx': 31863, 'rank': 1, 'distance': 0.5540807},
 1741: {'idx': 91347, 'rank': 1, 'distance': 0.56371224},
 203: {'idx': 50412, 'rank': 3, 'distance': 0.5652811},
 1844: {'idx': 31864, 'rank': 2, 'distance': 0.567515},
 1790: {'idx': 20656, 'rank': 2, 'distance': 0.5709613},
 571: {'idx': 53132, 'rank': 2, 'distance': 0.57630384},
 1254: {'idx': 57326, 'rank': 1, 'distan

In [ ]:
import numpy as np


In [ ]:
%%timeit

# downside here is that it's difficult to control which points are retrieved, can't simply increase K if that datapoint
# was already retrieved. would need to perform a full ranking over the whole query dataset, then get the kth
# (globally unique) datapoint with a reduction over the retrieved matrix starting from rank 1 / going sequentially like
# in the above loop
choices = np.random.choice(len(mutualplus_train_ctx), 2000, replace=False).tolist()
mmlu_db.index.search(mutualplus_db.index.reconstruct_batch(choices), 1)


7.47 s ± 488 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

choices = np.random.choice(len(mutualplus_train_ctx), 2000, replace=False).tolist()
res = [
    mmlu_db.similarity_search_with_score_by_vector(
        mutualplus_db.index.reconstruct(i).tolist()
    )[0]
    for i in choices
]
sorted(res, key=lambda k: k[1])


32.2 s ± 1.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%time

full_ranking = mmlu_db.index.search(
    mutualplus_db.index.reconstruct_batch(range(mutualplus_db.index.ntotal)),
    mutualplus_db.index.ntotal,
)
# BLAZING FAST, and it's on CPU


CPU times: user 1min 48s, sys: 13.3 s, total: 2min 2s
Wall time: 38.8 s


In [ ]:
np.unique(full_ranking[1][:100, 0]).shape


(54,)